<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-001.png" class="img-responsive"/>
</div>
<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-002.png" class="img-responsive"/>
</div>

In [15]:
l = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for i in l:
  print(i + 5)

5
6
7
8
9
10
11
12
13
14


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-003.png" class="img-responsive"/>
</div>



In [16]:
#print(5 + '5')

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-004.png" class="img-responsive"/>
</div>

In [36]:
combined = 5 + int('5')
print(type(combined))
print(combined)

<class 'int'>
10


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-005.png" class="img-responsive"/>
</div>

In [ ]:
def count_consecutive_pairs(ids, counts=None):

    # TODO

    return counts

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture03/slide-006.png" class="img-responsive"/>
</div>

In [7]:
!pwd
%cd ./sample_data/
!pwd
%cd /content/
!pwd

/root
[Errno 2] No such file or directory: './sample_data/'
/root
/root
/content
/content


In [ ]:
import os

try:
    import google.colab
    REPO_URL = "https://github.com/wtheisen/nd-cse-10124-lectures.git"

    REPO_NAME = "/content/nd-cse-10124-lectures"
    L_PATH = "nd-cse-10124-lectures/Datasets"

    %cd /content/
    !rm -r {REPO_NAME}

    # Clone repo
    if not os.path.exists(REPO_NAME):
        !git clone {REPO_URL}

        # cd into the data folder
        %cd {L_PATH}
        !pwd

except ImportError:
    print("Unable to download repo, either:")
    print("\tA.) You're not on colab")
    print("\tB.) It has already been cloned")

import utilities

/content
rm: cannot remove '/content/nd-cse-10124-lectures': No such file or directory
Cloning into 'nd-cse-10124-lectures'...


In [ ]:
class Cat():
    def __init__(self, color):
        self.color = color

    # TODO

In [39]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py
"""

class BasicTokenizer():
    def __init__(self):
        self.merges = {} # (int, int) -> int
        self.pattern = "" # str
        self.special_tokens = {} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = self._build_vocab() # int -> bytes

    def _build_vocab(self):
        # vocab is simply and deterministically derived from merges
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def _get_stats(self, ids, counts=None):
        """
        Given a list of integers, return a dictionary of counts of consecutive pairs
        Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
        Optionally allows to update an existing dictionary of counts
        """
        counts = {} if counts is None else counts
        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1
        return counts

    def _merge(self, ids, pair, idx):
        """
        In the list of integers (ids), replace all consecutive occurrences
        of pair with the new integer token idx
        Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
        """
        newids = []
        i = 0
        while i < len(ids):
            # if not at the very last position AND the pair matches, replace it
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes
        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = self._get_stats(ids)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = self._merge(ids, pair, idx)
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # save class variables
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

    def decode(self, ids):
        # given ids (list of integers), return Python string
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        # given a string text, return the token ids
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = self._get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = self._merge(ids, pair, idx)
        return ids

In [40]:
def get_file_str(source_file): # Define a function named 'get_lines_str' that takes an argument called 'source_file'
    zoomer_text = '' # Create a variable named 'lines' points to a list data structure
    with open(source_file) as f:
        for line in f: # For each item in the variable 'f', set the variable named 'line' equal to it, one by one
            zoomer_text += line.strip() # Add the content of the variable 'line' to the end of the list named 'lines' (after removing whitespace and newlines on either end)

    return zoomer_text # This function returns the variable named 'lines'

def get_lines_str(source_file): # Define a function named 'get_lines_str' that takes an argument called 'source_file'
    lines = [] # Create a variable named 'lines' points to a list data structure
    with open(source_file) as f:
        for line in f: # For each item in the variable 'f', set the variable named 'line' equal to it, one by one
            lines.append(line.strip()) # Add the content of the variable 'line' to the end of the list named 'lines' (after removing whitespace and newlines on either end)

    return lines # This function returns the variable named 'lines'

zoomer_text = get_file_str('zoomer.txt')
zoomer_lines = get_lines_str('zoomer.txt')

tokenizer = BasicTokenizer()
tokenizer.train(zoomer_text, 256 + 3)

for i in range(0, 7):
  print(zoomer_lines[i])
  tokenized_str = tokenizer.encode(zoomer_lines[i])
  print(tokenized_str)
  print(tokenizer.decode(tokenized_str))
  print('-' * 80)

Got the job today, big W!
[71, 111, 257, 116, 104, 256, 106, 111, 98, 258, 111, 100, 97, 121, 44, 32, 98, 105, 103, 32, 87, 33]
Got the job today, big W!
--------------------------------------------------------------------------------
I forgot my wallet at home, that’s an L.
[73, 32, 102, 111, 114, 103, 111, 257, 109, 121, 32, 119, 97, 108, 108, 101, 257, 97, 257, 104, 111, 109, 101, 44, 258, 104, 97, 116, 226, 128, 153, 115, 32, 97, 110, 32, 76, 46]
I forgot my wallet at home, that’s an L.
--------------------------------------------------------------------------------
Your tweet got 5 likes and 100 replies calling you out. L + ratio.
[89, 111, 117, 114, 258, 119, 101, 101, 257, 103, 111, 257, 53, 32, 108, 105, 107, 101, 115, 32, 97, 110, 100, 32, 49, 48, 48, 32, 114, 101, 112, 108, 105, 101, 115, 32, 99, 97, 108, 108, 105, 110, 103, 32, 121, 111, 117, 32, 111, 117, 116, 46, 32, 76, 32, 43, 32, 114, 97, 116, 105, 111, 46]
Your tweet got 5 likes and 100 replies calling you out. L + rat

In [48]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py

Unlike BasicTokenizer:
- RegexTokenizer handles an optional regex splitting pattern.
- RegexTokenizer handles optional special tokens.
"""

import regex as re


# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


class RegexTokenizer():
    def __init__(self, pattern=None):
        """
        - pattern: optional string to override the default (GPT-4 split pattern)
        - special_tokens: str -> int dictionary of special tokens
          example: {'<|endoftext|>': 100257}
        """
        self.merges = {} # (int, int) -> int
        self.pattern = "" # str
        self.special_tokens = {} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = self._build_vocab() # int -> bytes
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)
        self.special_tokens = {}
        self.inverse_special_tokens = {}

    def _build_vocab(self):
        # vocab is simply and deterministically derived from merges
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def _get_stats(self, ids, counts=None):
        """
        Given a list of integers, return a dictionary of counts of consecutive pairs
        Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
        Optionally allows to update an existing dictionary of counts
        """
        counts = {} if counts is None else counts
        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1
        return counts

    def _merge(self, ids, pair, idx):
        """
        In the list of integers (ids), replace all consecutive occurrences
        of pair with the new integer token idx
        Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
        """
        newids = []
        i = 0
        while i < len(ids):
            # if not at the very last position AND the pair matches, replace it
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1
        return newids

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # split the text up into text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
        for i in range(num_merges):
            # count the number of times every consecutive pair appears
            stats = {}
            for chunk_ids in ids:
                # passing in stats will update it in place, adding up counts
                self._get_stats(chunk_ids, stats)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = [self._merge(chunk_ids, pair, idx) for chunk_ids in ids]
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # save class variables
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

    def register_special_tokens(self, special_tokens):
        # special_tokens is a dictionary of str -> int
        # example: {"<|endoftext|>": 100257}
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}

    def decode(self, ids):
        # given ids (list of integers), return Python string
        part_bytes = []
        for idx in ids:
            if idx in self.vocab:
                part_bytes.append(self.vocab[idx])
            elif idx in self.inverse_special_tokens:
                part_bytes.append(self.inverse_special_tokens[idx].encode("utf-8"))
            else:
                raise ValueError(f"invalid token id: {idx}")
        text_bytes = b"".join(part_bytes)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def _encode_chunk(self, text_bytes):
        # return the token ids
        # let's begin. first, convert all bytes to integers in range 0..255
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = self._get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = self._merge(ids, pair, idx)
        return ids

    def encode_ordinary(self, text):
        """Encoding that ignores any special tokens."""
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8") # raw bytes
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids

    def encode(self, text, allowed_special="none_raise"):
        """
        Unlike encode_ordinary, this function handles special tokens.
        allowed_special: can be "all"|"none"|"none_raise" or a custom set of special tokens
        if none_raise, then an error is raised if any special token is encountered in text
        this is the default tiktoken behavior right now as well
        any other behavior is either annoying, or a major footgun
        """
        # decode the user desire w.r.t. handling of special tokens
        special = None
        if allowed_special == "all":
            special = self.special_tokens
        elif allowed_special == "none":
            special = {}
        elif allowed_special == "none_raise":
            special = {}
            assert all(token not in text for token in self.special_tokens)
        elif isinstance(allowed_special, set):
            special = {k: v for k, v in self.special_tokens.items() if k in allowed_special}
        else:
            raise ValueError(f"allowed_special={allowed_special} not understood")
        if not special:
            # shortcut: if no special tokens, just use the ordinary encoding
            return self.encode_ordinary(text)
        # otherwise, we have to be careful with potential special tokens in text
        # we handle special tokens by splitting the text
        # based on the occurrence of any exact match with any of the special tokens
        # we can use re.split for this. note that surrounding the pattern with ()
        # makes it into a capturing group, so the special tokens will be included
        special_pattern = "(" + "|".join(re.escape(k) for k in special) + ")"
        special_chunks = re.split(special_pattern, text)
        # now all the special characters are separated from the rest of the text
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for part in special_chunks:
            if part in special:
                # this is a special token, encode it separately as a special case
                ids.append(special[part])
            else:
                # this is an ordinary sequence, encode it normally
                ids.extend(self.encode_ordinary(part))
        return ids

    def save_vocab(self, vocab_file):
        # just for visualization purposes let's output the tokens
        # in the exact same format as the base class would.

        # build vocab being mindful of the byte shuffle
        vocab = {idx: bytes([self.inverse_byte_shuffle[idx]]) for idx in range(256)}

        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]

        # now merge the shuffled bytes and write to file
        inverted_merges = {idx: pair for pair, idx in self.merges.items()}

        with open(vocab_file, "w", encoding="utf-8") as f:
            for idx, token in vocab.items():
                s = render_token(token)

                if idx in inverted_merges:
                    idx0, idx1 = inverted_merges[idx]
                    s0 = render_token(vocab[idx0])
                    s1 = render_token(vocab[idx1])

                    f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
                else:
                    f.write(f"[{s}] {idx}\n")

In [49]:


zoomer_text = get_file_str('zoomer.txt')
zoomer_lines = get_lines_str('zoomer.txt')

regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(zoomer_text, 256 + 3)

for i in range(0, 7):
  print(zoomer_lines[i])
  tokenized_str = regex_tokenizer.encode(zoomer_lines[i])
  print(tokenized_str)
  print(regex_tokenizer.decode(tokenized_str))
  print('-' * 80)

Got the job today, big W!
[71, 111, 116, 256, 104, 101, 32, 106, 111, 98, 256, 111, 100, 97, 121, 44, 32, 98, 105, 103, 32, 87, 33]
Got the job today, big W!
--------------------------------------------------------------------------------
I forgot my wallet at home, that’s an L.
[73, 32, 102, 111, 114, 103, 111, 116, 32, 109, 121, 32, 119, 97, 108, 108, 101, 116, 32, 97, 116, 32, 104, 111, 109, 101, 44, 256, 104, 97, 116, 258, 115, 32, 97, 110, 32, 76, 46]
I forgot my wallet at home, that’s an L.
--------------------------------------------------------------------------------
Your tweet got 5 likes and 100 replies calling you out. L + ratio.
[89, 111, 117, 114, 256, 119, 101, 101, 116, 32, 103, 111, 116, 32, 53, 32, 108, 105, 107, 101, 115, 32, 97, 110, 100, 32, 49, 48, 48, 32, 114, 101, 112, 108, 105, 101, 115, 32, 99, 97, 108, 108, 105, 110, 103, 32, 121, 111, 117, 32, 111, 117, 116, 46, 32, 76, 32, 43, 32, 114, 97, 116, 105, 111, 46]
Your tweet got 5 likes and 100 replies calling yo

## Export to HTML

Uncomment the final line of the cell below and run it to export this notebook to HTML

In [ ]:
import os, json

def export_notebook():
  L_PATH = "nd-cse-10124-lectures/Notebooks"
  L = "Lecture_03_Tokenization"

  try:
      from google.colab import _message, files

      # where you WANT it to live (repo folder)
      repo_ipynb_path = f"/content/{L_PATH}/{L}.ipynb"

      # grab current notebook contents from the UI
      nb = _message.blocking_request("get_ipynb", timeout_sec=1)["ipynb"]

      # write it into the repo folder as a real file
      os.makedirs(os.path.dirname(repo_ipynb_path), exist_ok=True)
      with open(repo_ipynb_path, "w", encoding="utf-8") as f:
          json.dump(nb, f)

      # convert + download pdf
      !jupyter nbconvert --to html "{repo_ipynb_path}"
      files.download(repo_ipynb_path.replace(".ipynb", ".html"))
  except:
      import subprocess

      nb_fp = os.getcwd() + f'{L}.ipynb'
      print(os.getcwd())

      subprocess.run(["jupyter", "nbconvert", "--to", "html", nb_fp], check=True)

#export_notebook()

In [64]:
!pip install rustbpe

# -----------------------------------------------------------------------------
# Tokenizer based on rustbpe + tiktoken combo
import os
import copy
import pickle
import rustbpe
import tiktoken

from functools import lru_cache

SPECIAL_TOKENS = [
    "<|bos|>", # every document begins with the Beginning of Sequence (BOS) token that delimits documents
    # tokens below are only used during finetuning to render Conversations into token ids
    "<|user_start|>", # user messages
    "<|user_end|>",
    "<|assistant_start|>", # assistant messages
    "<|assistant_end|>",
    "<|python_start|>", # assistant invokes python REPL tool
    "<|python_end|>",
    "<|output_start|>", # python REPL outputs back to assistant
    "<|output_end|>",
]

SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class RustBPETokenizer:
    def __init__(self, enc, bos_token):
        self.enc = enc
        self.bos_token_id = self.encode_special(bos_token)

    @classmethod
    def train_from_iterator(cls, text_iterator, vocab_size):
        # 1) train using rustbpe
        tokenizer = rustbpe.Tokenizer()

        # the special tokens are inserted later in __init__, we don't train them here
        vocab_size_no_special = vocab_size - len(SPECIAL_TOKENS)
        assert vocab_size_no_special >= 256, f"vocab_size_no_special must be at least 256, got {vocab_size_no_special}"
        tokenizer.train_from_iterator(text_iterator, vocab_size_no_special, pattern=SPLIT_PATTERN)

        # 2) construct the associated tiktoken encoding for inference
        pattern = tokenizer.get_pattern()
        mergeable_ranks_list = tokenizer.get_mergeable_ranks()
        mergeable_ranks = {bytes(k): v for k, v in mergeable_ranks_list}
        tokens_offset = len(mergeable_ranks)
        special_tokens = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}

        enc = tiktoken.Encoding(name="rustbpe", pat_str=pattern, mergeable_ranks=mergeable_ranks, special_tokens=special_tokens)

        return cls(enc, "<|bos|>")

    # def get_vocab_size(self):
    #     return self.enc.n_vocab

    # def get_special_tokens(self):
    #     return self.enc.special_tokens_set

    # def id_to_token(self, id):
    #     return self.enc.decode([id])

    def encode_special(self, text):
        return self.enc.encode_single_token(text)

    def get_bos_token_id(self):
        return self.bos_token_id

    def encode(self, text, num_threads=8):
        ids = self.enc.encode_ordinary(text)

        return ids

    def decode(self, ids):
        return self.enc.decode(ids)

    def render_conversation(self, conversation, max_tokens=2048):
        """
        Tokenize a single Chat conversation (which we call a "doc" or "document" here).
        Returns:
        - ids: list[int] is a list of token ids of this rendered conversation
        - mask: list[int] of same length, mask = 1 for tokens that the Assistant is expected to train on.
        """
        # ids, masks that we will return and a helper function to help build them up.
        ids, mask = [], []
        def add_tokens(token_ids, mask_val):
            if isinstance(token_ids, int):
                token_ids = [token_ids]
            ids.extend(token_ids)
            mask.extend([mask_val] * len(token_ids))

        messages = conversation["messages"]

        # fetch all the special tokens we need
        bos = self.get_bos_token_id()
        user_start, user_end = self.encode_special("<|user_start|>"), self.encode_special("<|user_end|>")
        assistant_start, assistant_end = self.encode_special("<|assistant_start|>"), self.encode_special("<|assistant_end|>")
        python_start, python_end = self.encode_special("<|python_start|>"), self.encode_special("<|python_end|>")
        output_start, output_end = self.encode_special("<|output_start|>"), self.encode_special("<|output_end|>")

        # now we can tokenize the conversation
        add_tokens(bos, 0)
        for i, message in enumerate(messages):
            content = message["content"]

            if message["role"] == "user":
                value_ids = self.encode(content)
                add_tokens(user_start, 0)
                add_tokens(value_ids, 0)
                add_tokens(user_end, 0)
            elif message["role"] == "assistant":
                add_tokens(assistant_start, 0)
                # simple string => simply add the tokens
                value_ids = self.encode(content)
                add_tokens(value_ids, 1)

                add_tokens(assistant_end, 1)

        # truncate to max_tokens tokens MAX (helps prevent OOMs)
        ids = ids[:max_tokens]
        mask = mask[:max_tokens]
        return ids, mask

    def visualize_tokenization(self, ids, mask, with_token_id=False):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'
        tokens = []
        for i, (token_id, mask_val) in enumerate(zip(ids, mask)):
            token_str = self.decode([token_id])
            color = GREEN if mask_val == 1 else RED
            tokens.append(f"{color}{token_str}{RESET}")
            if with_token_id:
                tokens.append(f"{GRAY}({token_id}){RESET}")
        return '|'.join(tokens)

In [67]:
zoomer_text = get_file_str('zoomer.txt')

nanochat_tokenizer = RustBPETokenizer.train_from_iterator(zoomer_text, 256 + len(SPECIAL_TOKENS))

conversation = {
    'messages': [
        {'role': 'user', 'content': 'Hello there general kenobi'},
        {'role': 'assistant', 'content': 'Hello there grevious'},
        {'role': 'user', 'content': 'I love neffer'},
    ]
}

ids, masks = nanochat_tokenizer.render_conversation(conversation)
print(nanochat_tokenizer.visualize_tokenization(ids, masks, with_token_id=True))

<|bos|>|(256)|<|user_start|>|(257)|H|(72)|e|(101)|l|(108)|l|(108)|o|(111)| |(32)|t|(116)|h|(104)|e|(101)|r|(114)|e|(101)| |(32)|g|(103)|e|(101)|n|(110)|e|(101)|r|(114)|a|(97)|l|(108)| |(32)|k|(107)|e|(101)|n|(110)|o|(111)|b|(98)|i|(105)|<|user_end|>|(258)|<|assistant_start|>|(259)|H|(72)|e|(101)|l|(108)|l|(108)|o|(111)| |(32)|t|(116)|h|(104)|e|(101)|r|(114)|e|(101)| |(32)|g|(103)|r|(114)|e|(101)|v|(118)|i|(105)|o|(111)|u|(117)|s|(115)|<|assistant_end|>|(260)|<|user_start|>|(257)|I|(73)| |(32)|l|(108)|o|(111)|v|(118)|e|(101)| |(32)|n|(110)|e|(101)|f|(102)|f|(102)|e|(101)|r|(114)|<|user_end|>|(258)
